# Merging - Filtering

We shall do the following tasks in this notebook:
0. Read original dataset.
1. Correct the merging process, and remove papers which were not merged to MAG properly.
2. Check/Validate that the authors extracted from MAG are actually retracted authors using fuzzy matching.
3. Remove authors with multiple offences.
4. Filter papers retracted in bulk.
5. Filter papers retracted beyond 1990-2015. 
6. Remove papers with anomalies (date problems or author matching issues).
7. Removing authors without certain fields (affiliations, disciplines)


In [1]:
import pandas as pd
import numpy as np
from thefuzz import fuzz

In [2]:
indir = "../../data/main/"

In [3]:
# Let us read df_papers first
df_papers = pd.read_csv(indir+"RW_papers.csv")

df_papers.head(2)

# converting dates to proper format
df_papers['OriginalPaperDate'] = df_papers['OriginalPaperDate'].str.split().str[0]

# Converting Retraction Date to datetime format
df_papers['OriginalPaperDate'] = pd.to_datetime(df_papers['OriginalPaperDate'], 
                                                  format='%d/%m/%Y')
# Extracting year
df_papers['OriginalPaperYear'] = df_papers['OriginalPaperDate'].dt.year

# Removing an anomaous entry of 1976
df_papers = df_papers[~df_papers.RetractionDate.eq('6/24/1756 12:00:00 AM')].copy()

df_papers['RetractionDate'] = df_papers['RetractionDate'].str.split().str[0]

# Converting Retraction Date to datetime format
df_papers['RetractionDate'] = pd.to_datetime(df_papers['RetractionDate'], 
                                               format='%d/%m/%Y')

df_papers['RetractionYear'] = df_papers['RetractionDate'].dt.year

df_papers.columns

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (12,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Record ID', 'Title', 'Subject', 'Institution', 'Journal', 'Publisher',
       'Country', 'Author', 'URLS', 'ArticleType', 'RetractionDate',
       'RetractionDOI', 'RetractionPubMedID', 'OriginalPaperDate',
       'OriginalPaperDOI', 'OriginalPaperPubMedID', 'RetractionNature',
       'Reason', 'Paywalled', 'Notes', 'RWMAGFuzzyMatchMakerScore',
       'MAGPIDMatchedByFuzzy', 'MAGTitleMatchedByFuzzy',
       'MAGYearMatchedByFuzzy', 'RWMAGMatchedBy', 'RWMAGSameWordsRatio',
       'RWMAGFuzzyWuzzyScore', 'RWMAGFuzzyWuzzyPartialScore',
       'MAGPIDMatchedByDOI', 'MAGTitleMatchedByDOI', 'MAGYearMatchedByDOI',
       'MAGPID', 'RWJournalNorm', 'MAGJCNameByPID', 'MAGJCIDByPID',
       'MAGJournalType', 'MAGJCID', 'MAGJCName', 'OriginalPaperYear',
       'RetractionYear'],
      dtype='object')

In [4]:
# Creating Matchedby column
def mergedby(row):
    if pd.isnull(row['MAGTitleMatchedByDOI']):
        assert(not pd.isnull(row['MAGTitleMatchedByFuzzy']))
        return "Title"
    elif pd.isnull(row['MAGTitleMatchedByFuzzy']):
        assert(not pd.isnull(row['MAGTitleMatchedByDOI']))
        return "DOI"

df_papers['MergingMethod'] = df_papers.apply(lambda row: mergedby(row), axis=1)

In [5]:
def compute_fuzz_ratio(row, ratioType='full'):
    if pd.isnull(row['MAGTitleMatchedByDOI']):
        assert(not pd.isnull(row['MAGTitleMatchedByFuzzy']))
        if ratioType == 'partial':
            return fuzz.partial_ratio(row['MAGTitleMatchedByFuzzy'],row['Title'].lower())
        else:
            return fuzz.ratio(row['MAGTitleMatchedByFuzzy'],row['Title'].lower())
    
    elif pd.isnull(row['MAGTitleMatchedByFuzzy']):
        assert(not pd.isnull(row['MAGTitleMatchedByDOI']))
        if ratioType == 'partial':
            return fuzz.partial_ratio(row['MAGTitleMatchedByDOI'],row['Title'].lower())
        else:
            return fuzz.ratio(row['MAGTitleMatchedByDOI'],row['Title'].lower())

df_papers['FuzzRatio'] = df_papers.apply(lambda row: compute_fuzz_ratio(row), axis=1)

df_papers['PartialFuzzRatio'] = df_papers.apply(lambda row: compute_fuzz_ratio(row,
                                                                    ratioType='partial'), axis=1)

df_papers.head(2)

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,...,MAGJCNameByPID,MAGJCIDByPID,MAGJournalType,MAGJCID,MAGJCName,OriginalPaperYear,RetractionYear,MergingMethod,FuzzRatio,PartialFuzzRatio
0,28599,TWEAK-Fn14 Influences Neurogenesis Status via ...,(BLS) Biochemistry;(BLS) Biology - Cellular;(B...,Department of Otolaryngology-Head and Neck Sur...,Molecular Neurobiology,Springer,China,Jing Xu;Jian He;Huang He;Renjun Peng;Jian Xi,NaN,Research Article;,...,molecular neurobiology,194681035.0,journal,194681035.0,molecular neurobiology,2016.0,2021,Title,96,96
1,28504,Efficacy of vitamin C for the prevention and t...,(HSC) Medicine - Infectious Disease;(HSC) Medi...,"Department of General Practice, Clermont Auver...",European Journal of Clinical Pharmacology,Springer,France,Philippe Vorilhon;Bastien Arpajou;HÃ©lÃ¨ne Vai...,NaN,Meta-Analysis;,...,european journal of clinical pharmacology,31780008.0,journal,31780008.0,european journal of clinical pharmacology,2018.0,2021,Title,99,98


In [6]:
# These articles are matched to wrong records in MAG and need to be removed.
wrongLst_validation = [446,9042,16825]
wrongLst_correction = [3524,17581,2412,
                      3137,7893,4158,3648,
                      3160,5821,16936,
                      2264,5371,16705,
                      4565,2659,6487,
                      16197, 18696,22181,
                      919, 1517, 25026, 17617,
                      1779, 6229, 6489, 23072, 
                      5402, 2498, 2748, 22371,
                      4517, 6486, 10347, 5710, 
                      9860, 8518, 4883, 3852, 
                      16943, 28, 6471, 17127, 4861,
                      2715, 17736, 4566, 2756, 1791, 
                      4969, 18913, 4379, 5845, 6122,
                      3731, 3051, 8129, 19452, 18536,
                      16942, 2812, 4783, 1734, 14819, 
                      18818, 15305, 8001, 1919, 3436, 
                      4174, 492, 16721,]

df_papers_filtered = df_papers[~df_papers['Record ID'].\
                                   isin(wrongLst_correction+wrongLst_validation)]


"""
Note: This number is the number of papers after merging and filtering based on manual checking. 
But this number includes the papers to be removed for anomalies.
"""

df_papers_filtered['Record ID'].nunique()

24187

In [7]:
# Now we read df_authors
df_authors = pd.read_csv(indir+"RW_authors_method2_nonanomalous_w_fields.csv")


"""
Note: This number includes the number of papers after merging and filtering based on manual checking.
It also includes the papers removed for anomalies. But it does not include papers 
to be removed for papers preceded by "retraction notices". It, however, does include papers 
with authors with no disciplines, and does not include papers beyond 1990-2020. These 
were earlier filters.
"""
print(df_authors[~df_authors['Record ID'].isin(wrongLst_correction+wrongLst_validation)]\
            [['Record ID']].drop_duplicates().shape)


df_authors = df_authors[~df_authors['Record ID'].isin(wrongLst_correction+wrongLst_validation)]

"""
Now let us merge df_authors and df_papers to (a) get fuzzy scores for each title, and 
(b) also to compare the author names between MAG and RW for papers whose MAG titles 
have a "retraction notice" as a prefix. But to be able to do that, we would need 
author full names which we can extract from ethnicity file as we used names to 
extract ethnicity.
"""

df_authors.columns

(23269, 1)


Index(['Record ID', 'MAGPID', 'RetractionYear', 'OriginalPaperYear',
       'MAGAffID', 'MAGRetractedPIDYear', 'MAGAffiliationRank', 'MAGFirstName',
       'GenderizeGender', 'GenderizeConfidence', 'GenderizeCount',
       'FinalGender', 'MAGFirstPubYear', 'OriginalPaperYearMethod1',
       'OriginalPaperYearMethod2', 'MAGCumPapersYear', 'MAGCumPapers',
       'MAGCumCitationsYear', 'MAGCumCitations', 'MAGCumCollaboratorsYear',
       'MAGCumCollaborators', 'MAGCumCollaborationsYear',
       'MAGCumCollaborations', 'MAGFirstAffID', 'MAGFirstYear',
       'MAGFirstAffiliationRank', 'MAGAIDSequence', 'MAGAIDSequenceMax',
       'MAGFirstAuthorFlag', 'MAGLastAuthorFlag', 'MAGRetractionYearAffID',
       'MAGRetractionYearAffYear', 'MAGRetractionYearAffRank', 'MAGAID',
       'MAGrootFID', 'MAGrootFIDMaxPercent'],
      dtype='object')

In [8]:
df_authors.to_csv(indir+"/merged_sample.csv",index=False)

In [8]:
df_papers_relevant = df_papers_filtered[['Record ID','MAGPID','Title','MAGJCID','Author',
                                        'MAGTitleMatchedByFuzzy','FuzzRatio',
                                        'OriginalPaperDOI','MAGTitleMatchedByDOI']].\
                                        drop_duplicates()

df_papers_authors = df_authors.merge(df_papers_relevant, on=['Record ID','MAGPID'], how='left')
print("Merged Shape:",df_papers_authors.shape)

# Let us now extract names of authors
# For that we will read the ethnicity file
df_author_names = pd.read_csv("../../data/ethnicity/MAGAID_ethnicity_retracted.csv",
                             usecols=['MAGAID','NormalizedName'])

df_papers_authors_w_names = df_papers_authors.merge(df_author_names, on='MAGAID')
df_papers_authors_w_names.columns, df_papers_authors_w_names['MAGAID'].nunique()

Merged Shape: (210041, 43)


(Index(['Record ID', 'MAGPID', 'RetractionYear', 'OriginalPaperYear',
        'MAGAffID', 'MAGRetractedPIDYear', 'MAGAffiliationRank', 'MAGFirstName',
        'GenderizeGender', 'GenderizeConfidence', 'GenderizeCount',
        'FinalGender', 'MAGFirstPubYear', 'OriginalPaperYearMethod1',
        'OriginalPaperYearMethod2', 'MAGCumPapersYear', 'MAGCumPapers',
        'MAGCumCitationsYear', 'MAGCumCitations', 'MAGCumCollaboratorsYear',
        'MAGCumCollaborators', 'MAGCumCollaborationsYear',
        'MAGCumCollaborations', 'MAGFirstAffID', 'MAGFirstYear',
        'MAGFirstAffiliationRank', 'MAGAIDSequence', 'MAGAIDSequenceMax',
        'MAGFirstAuthorFlag', 'MAGLastAuthorFlag', 'MAGRetractionYearAffID',
        'MAGRetractionYearAffYear', 'MAGRetractionYearAffRank', 'MAGAID',
        'MAGrootFID', 'MAGrootFIDMaxPercent', 'Title', 'MAGJCID', 'Author',
        'MAGTitleMatchedByFuzzy', 'FuzzRatio', 'OriginalPaperDOI',
        'MAGTitleMatchedByDOI', 'NormalizedName'],
       dtype='objec

In [9]:
# Let us now do validation to check that author names match
"""
The reason we are doing this validation is because many papers we have merged 
in MAG have a prefix "retraction notice", and we want to check if we are 
merging right because a retraction notice could be from an editor, and 
so we would then be studying the editor instead of the retracted author.
"""

df_validation = df_papers_authors_w_names[~df_papers_authors_w_names.MAGTitleMatchedByFuzzy.isna() & 
                                        ~df_papers_authors_w_names.Author.isna() & 
                                        ~df_papers_authors_w_names.NormalizedName.isna()]\
    [['Record ID','Title','MAGTitleMatchedByFuzzy',
      'FuzzRatio','Author','NormalizedName','RetractionYear','MAGAID']].drop_duplicates()

def compute_author_partial_fuzz_ratio(row):
    ratio1 = fuzz.partial_ratio(row['Author'].lower(), row['NormalizedName'])
    ratio2 = 0
    normalizedName = row['NormalizedName'].split()
    normalizedName.reverse()
    if(ratio1 < 90):
        ratio2 = fuzz.partial_ratio(row['Author'].lower(), 
                                    " ".join(normalizedName))
    return max(ratio1, ratio2)

df_validation['AuthorFuzzRatio'] = df_validation.\
                    apply(lambda row:compute_author_partial_fuzz_ratio(row),
                    axis=1)

def contains_prefix(row):
    if 'notice of retraction' in row['MAGTitleMatchedByFuzzy']:
        return True
    if 'retraction notice' in row['MAGTitleMatchedByFuzzy']:
        return True
    if 'retraction' in row['MAGTitleMatchedByFuzzy']:
        return True
    else:
        return False

df_validation['ContainsPrefix'] = df_validation.apply(lambda row: contains_prefix(row),
                                                     axis=1)

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

lst_rids_validated = df_validation[df_validation.AuthorFuzzRatio.ge(80)]['Record ID']

df_validation[~df_validation['Record ID'].isin(lst_rids_validated) & 
              df_validation.RetractionYear.isin(range(1990,2016))][['Author','NormalizedName','MAGAID',
                                                                   'AuthorFuzzRatio','Record ID']]\
                ['Record ID'].nunique()


220

In [11]:
df_papers_authors_w_names['Record ID']

23269

In [12]:
## Let us first remove authors that occur multiple times

df1_temp_1 = df_papers_authors_w_names.groupby('MAGAID')['MAGPID'].nunique().\
                    reset_index().rename(columns={'MAGPID':'NumMAGPIDRetractedByMAGAID'})

df1_temp_2 = df_papers_authors_w_names.groupby('MAGAID')['Record ID'].nunique().\
                reset_index().rename(columns={'Record ID':'NumRecordsRetractedByMAGAID'})


lst_offenders = df1_temp_1[df1_temp_1.NumMAGPIDRetractedByMAGAID.gt(1)].MAGAID.unique().tolist() + \
                df1_temp_2[df1_temp_2.NumRecordsRetractedByMAGAID.gt(1)].MAGAID.unique().tolist()

# Now let us remove the offenders from the list

df_authors_f1 = df_papers_authors_w_names[~df_papers_authors_w_names.MAGAID.isin(lst_offenders)]

print("Number of unique authors after removing repeated offenders:", df_authors_f1.MAGAID.unique().shape[0])
print("Number of papers after removing repeated offenders:", df_authors_f1['Record ID'].nunique())
df_papers_authors_w_names.MAGAID.nunique()

Number of unique authors after removing repeated offenders: 64995
Number of papers after removing repeated offenders: 20338


72594

In [49]:
# Let us look at the rows removed. 
df_papers_authors_w_names[df_papers_authors_w_names.MAGAID.isin(lst_offenders)][['Record ID','MAGAID']]\
    .drop_duplicates().groupby('MAGAID')['Record ID'].nunique()

df_papers_authors_w_names[df_papers_authors_w_names.MAGAID.isin(lst_offenders)]\
    [['Record ID', 'MAGAID', 'MAGPID']].drop_duplicates().groupby('MAGAID').count().sort_values(by='Record ID')

,Record ID,MAGPID
MAGAID,,
1.939220e+05,2,2
2.306160e+09,2,2
2.306148e+09,2,2
2.306114e+09,2,2
2.306114e+09,2,2
...,...,...
2.156775e+09,62,62
2.051380e+09,74,74
2.128683e+09,82,82


In [57]:
# Let us save the list of offenders for our analysis later
df_papers_authors_w_names[df_papers_authors_w_names.MAGAID.isin(lst_offenders)]\
        [['MAGAID','Record ID','MAGPID','Title']].drop_duplicates().\
        to_csv(indir+"/authors_w_multiple_retractions.csv", index=False)

In [11]:
## Removing papers that retracted in bulk
df2_temp = df_papers.groupby(['MAGJCID','RetractionDate'])['MAGPID'].nunique().reset_index().\
            rename(columns={'MAGPID':'NumPapersRetractedInBulk'}).sort_values(by='NumPapersRetractedInBulk')

# Merging the above with papers
df_papers_merged = df_papers.merge(df2_temp, on=['MAGJCID','RetractionDate'])

# These are the valid papers after this filter
lst_valid_papers = df_papers_merged[df_papers_merged.NumPapersRetractedInBulk.le(5)].MAGPID.unique().tolist()

# Now let us create a new data frame for authors with only these papers

df_authors_f2 = df_authors_f1[df_authors_f1.MAGPID.isin(lst_valid_papers)]

print("Papers left:",df_authors_f2['Record ID'].nunique())
print("Authors left:",df_authors_f2['MAGAID'].nunique())

Papers left: 10355
Authors left: 41430


In [12]:
# Now let us remove papers beyond [1990,2015]

df_authors_f3 = df_authors_f2[df_authors_f2.RetractionYear.le(2015) & \
                                  df_authors_f2.RetractionYear.ge(1990)]

print("Papers left:",df_authors_f3['Record ID'].nunique())
print("Authors left:",df_authors_f3['MAGAID'].nunique())

Papers left: 5291
Authors left: 19769


In [13]:
# Let us do some sensiblity checks

(df_authors_f3['RetractionYear']-df_authors_f3['OriginalPaperYearMethod2']).describe()

count    44512.000000
mean         1.944195
std          2.877742
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max         35.000000
dtype: float64

In [14]:
(df_authors_f3['RetractionYear']-df_authors_f3['MAGRetractionYearAffYear']).describe()

count    37567.000000
mean         0.926026
std          2.162793
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         45.000000
dtype: float64

In [15]:
(df_authors_f3['RetractionYear']-df_authors_f3['MAGFirstYear']).describe()

count    37567.000000
mean         8.245481
std         10.100575
min          0.000000
25%          1.000000
50%          4.000000
75%         12.000000
max        213.000000
dtype: float64

In [18]:
df_authors_f3.MAGAID.columns

Index(['Record ID', 'MAGPID', 'RetractionYear', 'OriginalPaperYear',
       'MAGAffID', 'MAGRetractedPIDYear', 'MAGAffiliationRank', 'MAGFirstName',
       'GenderizeGender', 'GenderizeConfidence', 'GenderizeCount',
       'FinalGender', 'MAGFirstPubYear', 'OriginalPaperYearMethod1',
       'OriginalPaperYearMethod2', 'MAGCumPapersYear', 'MAGCumPapers',
       'MAGCumCitationsYear', 'MAGCumCitations', 'MAGCumCollaboratorsYear',
       'MAGCumCollaborators', 'MAGCumCollaborationsYear',
       'MAGCumCollaborations', 'MAGFirstAffID', 'MAGFirstYear',
       'MAGFirstAffiliationRank', 'MAGAIDSequence', 'MAGAIDSequenceMax',
       'MAGFirstAuthorFlag', 'MAGLastAuthorFlag', 'MAGRetractionYearAffID',
       'MAGRetractionYearAffYear', 'MAGRetractionYearAffRank', 'MAGAID',
       'MAGrootFID', 'MAGrootFIDMaxPercent', 'Title', 'MAGJCID', 'Author',
       'MAGTitleMatchedByFuzzy', 'FuzzRatio', 'OriginalPaperDOI',
       'MAGTitleMatchedByDOI', 'NormalizedName'],
      dtype='object')

In [49]:
# Removing authors without first affiliation ID
df_authors_f4 = df_authors_f3[~df_authors_f3.MAGFirstAffID.isna()]

print("Papers left:",df_authors_f4['Record ID'].nunique())
print("Authors left:",df_authors_f4['MAGAID'].nunique())

Papers left: 4749
Authors left: 16757


In [50]:
# Removing authors without retraction year affiliation ID
df_authors_f5 = df_authors_f4[~df_authors_f4.MAGRetractionYearAffID.isna()]

print("Papers left:",df_authors_f5['Record ID'].nunique())
print("Authors left:",df_authors_f5['MAGAID'].nunique())

Papers left: 4749
Authors left: 16757


In [51]:
# Removing authors without retraction year affiliation ID
df_authors_f6 = df_authors_f5[~df_authors_f5.MAGrootFID.isna()]

print("Papers left:",df_authors_f6['Record ID'].nunique())
print("Authors left:",df_authors_f6['MAGAID'].nunique())

Papers left: 4747
Authors left: 16724


In [86]:
# Removing authors without retraction year affiliation ID
df_authors_f7 = df_authors_f6[df_authors_f6.GenderizeConfidence.gt(0.50)].copy()

print("Papers left:",df_authors_f7['Record ID'].nunique())
print("Authors left:",df_authors_f7['MAGAID'].nunique())

Papers left: 4675
Authors left: 15672


In [87]:
# Let us impute MAG Afffiliation Rank 1001- for those with no affiliation rank

df_authors_f7['MAGFirstAffiliationRank'] = \
    df_authors_f7['MAGFirstAffiliationRank'].fillna('1001-')

df_authors_f7['MAGRetractionYearAffRank'] = \
    df_authors_f7['MAGRetractionYearAffRank'].fillna('1001-')

In [96]:
df_authors_f7.to_csv(indir+"/filtered_sample.csv",index=False)